## Hardware Testbed and Large-scale Testbed Co-simulation

In [ ]:
# --- imports ---

import numpy as np
import pandas as pd

import matplotlib
matplotlib.rcParams.update(matplotlib.rcParamsDefault)

import matplotlib.pyplot as plt
import scienceplots

import os
import andes

import geopandas as gpd
import contextily as cx

from shapely.geometry import LineString

In [ ]:
%matplotlib inline

In [ ]:
path_proj = os.getcwd()
path_case = os.path.join(path_proj, 'case')
path_data = os.path.join(path_proj, 'data')
path_map = os.path.join(path_proj, 'map')

path_out = os.path.join(os.path.abspath('..'), 'output')

sscase = os.path.join(path_case, 'ieee39_htb.xlsx')

ss = andes.load(sscase,
                no_output=True,
                default_config=False,
                setup=False)

In [61]:
map_state_name = "cb_2018_us_state_500k.zip"
map_county_name = "cb_2018_us_county_within_cd116_500k.zip"
map_division_name = "cb_2018_us_division_500k.zip"

# Read in shapefile of US states
map_dv = gpd.read_file(os.path.join(path_map, map_division_name))
map_st = gpd.read_file(os.path.join(path_map, map_state_name))
map_ct = gpd.read_file(os.path.join(path_map, map_county_name))

# Filter the map data to only include the Northeast states
dv_ne_idx = ['1', '2']

dv_ne = map_dv[map_dv['GEOID'].isin(dv_ne_idx)]

In [68]:
map_dv

,DIVISIONCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,1,0300000US1,1,New England,69,162376417481,24072855206,"MULTIPOLYGON (((-67.32259 44.61160, -67.32174 ..."
1,2,0300000US2,2,Middle Atlantic,69,256981418064,26186444931,"MULTIPOLYGON (((-72.03683 41.24984, -72.03496 ..."
2,3,0300000US3,3,East North Central,69,629289745590,151252485939,"MULTIPOLYGON (((-82.73571 41.60336, -82.73392 ..."
3,4,0300000US4,4,West North Central,69,1314707528663,33020781573,"MULTIPOLYGON (((-89.59206 47.96668, -89.59147 ..."
4,5,0300000US5,5,South Atlantic,69,687099317084,71741527671,"MULTIPOLYGON (((-75.56555 39.51485, -75.56174 ..."
5,8,0300000US8,8,Mountain,69,2216504548727,20112368319,"POLYGON ((-120.00574 39.22866, -120.00559 39.2..."
6,9,0300000US9,9,Pacific,69,2319697198955,296474924140,"MULTIPOLYGON (((-147.17351 60.91154, -147.1700..."
7,6,0300000US6,6,East South Central,69,461789786924,13245708132,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
8,7,0300000US7,7,West South Central,69,1100982564361,49097374744,"MULTIPOLYGON (((-88.86770 29.86155, -88.86566 ..."


In [65]:
map_st.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ..."
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00257 36.52659, -103.00219 36.6..."
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ..."
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.64320 38.16909, -82.64300 38.169..."


In [62]:
map_st[map_st['AFFGEOID'].isin(dv_ne['AFFGEOID'])]

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry


In [64]:
map_st['AFFGEOID'].unique()

array(['0400000US28', '0400000US37', '0400000US40', '0400000US51',
       '0400000US54', '0400000US22', '0400000US26', '0400000US25',
       '0400000US16', '0400000US12', '0400000US31', '0400000US53',
       '0400000US35', '0400000US72', '0400000US46', '0400000US48',
       '0400000US06', '0400000US01', '0400000US13', '0400000US42',
       '0400000US29', '0400000US08', '0400000US49', '0400000US47',
       '0400000US56', '0400000US36', '0400000US20', '0400000US02',
       '0400000US32', '0400000US17', '0400000US50', '0400000US30',
       '0400000US19', '0400000US45', '0400000US33', '0400000US04',
       '0400000US11', '0400000US60', '0400000US78', '0400000US34',
       '0400000US24', '0400000US23', '0400000US15', '0400000US10',
       '0400000US66', '0400000US69', '0400000US44', '0400000US21',
       '0400000US39', '0400000US55', '0400000US41', '0400000US38',
       '0400000US05', '0400000US18', '0400000US27', '0400000US09'],
      dtype=object)

In [60]:
map_st.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,28,01779790,0400000US28,28,MS,Mississippi,00,121533519481,3926919758,"MULTIPOLYGON (((-88.50297 30.21523, -88.49176 ..."
1,37,01027616,0400000US37,37,NC,North Carolina,00,125923656064,13466071395,"MULTIPOLYGON (((-75.72681 35.93584, -75.71827 ..."
2,40,01102857,0400000US40,40,OK,Oklahoma,00,177662925723,3374587997,"POLYGON ((-103.00257 36.52659, -103.00219 36.6..."
3,51,01779803,0400000US51,51,VA,Virginia,00,102257717110,8528531774,"MULTIPOLYGON (((-75.74241 37.80835, -75.74151 ..."
4,54,01779805,0400000US54,54,WV,West Virginia,00,62266474513,489028543,"POLYGON ((-82.64320 38.16909, -82.64300 38.169..."


In [ ]:

dv_ne['GEOID'] = dv_ne['GEOID'].zfill(2)
st_ne = map_st[map_st['GEOID'].isin(dv_ne['GEOID'].str.zfill(2))]
ct_ne = map_ct[map_ct['GEOID'].str[:2].astype(int).astype(str).isin(dv_ne['GEOID'])]

In [ ]:
dv_ne

In [ ]:
map_st

In [ ]:
st_ne

In [ ]:
ct_ne

In [ ]:
ax = ct_ne.plot(facecolor='none', edgecolor='black', linewidth=0.5, linestyle='--')
dv_ne.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, alpha=0.5)

ax.set_xlim(-125, -65)
ax.set_ylim(25, 50)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 15))

# dv_ne.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, alpha=0.5)
ct_ne.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, linestyle=':')

for idx, row in dv_ne.iterrows():
    ax.annotate(text=row['NAME'], xy=row['geometry'].centroid.coords[0],
                ha='center', fontsize=10)

ax.set_xlim(-80, -65)
ax.set_ylim(40, 48)


In [ ]:
map_state_name = "cb_2018_us_state_500k.zip"
map_county_name = "cb_2018_us_county_within_cd116_500k.zip"
map_division_name = "cb_2018_us_division_500k.zip"

# Read in shapefile of US states
map_division = gpd.read_file(os.path.join(path_map, map_division_name))
map_county = gpd.read_file(os.path.join(path_map, map_county_name))
map_state = gpd.read_file(os.path.join(path_map, map_state_name))

# Filter the map data to only include the Northeast states
divisions_ne_idx = ['1', '2']
divisions_ne = map_division[map_division['GEOID'].isin(divisions_ne_idx)]

fig, ax = plt.subplots(figsize=(10, 15))

# divisions_ne.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5)

divisions_ne.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5)

for idx, row in divisions_ne.iterrows():
    ax.annotate(text=row['NAME'], xy=row['geometry'].centroid.coords[0],
                ha='center', fontsize=10)

ax.set_xlim(-80, -65)
ax.set_ylim(40, 48)

In [ ]:
map_division

In [ ]:
divisions_ne

In [ ]:
map_state[map_state['AFFGEOID'].isin(map_division)]

In [ ]:
map_county

In [ ]:
counties_ne = map_county[map_county.within(divisions_ne.unary_union)]

In [ ]:
# stateToRemove = ['02', '15', '72']
# usa_map = usa_map[~usa_map['STATEFP'].isin(stateToRemove)]

fig, ax = plt.subplots(figsize=(10, 15))

divisions_ne.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5)
counties_ne.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
# states_ne.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1, linestyle='--')
# map_county.plot(ax=ax)

for idx, row in divisions_ne.iterrows():
    ax.annotate(text=row['NAME'], xy=row['geometry'].centroid.coords[0],
                ha='center', fontsize=10)

ax.set_xlim(-95, -65)
ax.set_ylim(35, 50)

In [ ]:
map_name = "cb_2018_us_cd116_500k.shp"

# Read in shapefile of US states
usa_map = gpd.read_file(os.path.join(path_map, map_name))

# --- Bus Data ---
bus_gen = list(ss.PV.bus.v) + list(ss.Slack.bus.v)
bus_load = list(ss.PQ.bus.v)

xcoord_bus_gen = ss.Bus.get(src='xcoord', idx=bus_gen, attr='v')
ycoord_bus_gen = ss.Bus.get(src='ycoord', idx=bus_gen, attr='v')

xcoord_bus_load = ss.Bus.get(src='xcoord', idx=bus_load, attr='v')
ycoord_bus_load = ss.Bus.get(src='ycoord', idx=bus_load, attr='v')

p_bus = gpd.GeoDataFrame(geometry=gpd.points_from_xy(
    ss.Bus.xcoord.v, ss.Bus.ycoord.v))
p_bus_gen = gpd.GeoDataFrame(geometry=gpd.points_from_xy(
    xcoord_bus_gen, ycoord_bus_gen))
p_bus_load = gpd.GeoDataFrame(geometry=gpd.points_from_xy(
    xcoord_bus_load, ycoord_bus_load))

# --- Line Data ---

transmission_lines = []

line_bus1 = ss.Line.get(src='bus1', idx=ss.Line.idx.v, attr='v')
line_bus2 = ss.Line.get(src='bus2', idx=ss.Line.idx.v, attr='v')

for bus1, bus2 in zip(line_bus1, line_bus2):
    bus1x = ss.Bus.get(src='xcoord', idx=bus1, attr='v')
    bus1y = ss.Bus.get(src='ycoord', idx=bus1, attr='v')
    bus2x = ss.Bus.get(src='xcoord', idx=bus2, attr='v')
    bus2y = ss.Bus.get(src='ycoord', idx=bus2, attr='v')
    line_coords = [(bus1x, bus1y), (bus2x, bus2y)]
    transmission_lines.append(LineString(line_coords))

l_lines = gpd.GeoDataFrame(geometry=transmission_lines)

# --- Plotting ---
ax_map = usa_map.plot(figsize=(10, 6))
# cx.add_basemap(ax_map, source=cx.providers.Stamen.TonerLite)

ax, markersize, facecolor = ax_map, 20, 'none'
l_lines.plot(ax=ax, color='black', linewidth=1)
p_bus_gen.plot(ax=ax, markersize=markersize, color='red')
p_bus_load.plot(ax=ax, markersize=markersize, color='black')

# cx.add_basemap(ax=ax, crs=world.crs.to_string(),
#                source=cx.providers.Stamen.TonerLite)

# Get the extent of the points
marginx = 0.3
marginy = 0.15
x_min, y_min, x_max, y_max = p_bus.total_bounds
x_range = x_max - x_min
y_range = y_max - y_min
ax_map.set_xlim(x_min - marginx * x_range, x_max + marginx * x_range)
ax_map.set_ylim(y_min - marginy * y_range, y_max + marginy * y_range)

ax_map.set_axis_off()
ax_map.set_title('System topology')

# for name, x, y in zip(ss.Bus.name.v, ss.Bus.xcoord.v, ss.Bus.ycoord.v):
#     if 'LOAD' in name:
#         ax.annotate(name, ( x, y,),
#                     color='black', fontsize=8,
#                     ha='right', va='top')


In [ ]:
# Read shapefile and plot map
# world = gpd.read_file(gpd.datasets.get_path('nybb'))

usa_map = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa_map = usa_map[usa_map.name == 'United States']
usa_map.plot(ax=ax_map, edgecolor='black', facecolor='none')

# --- Bus Data ---
bus_gen = list(ss.PV.bus.v) + list(ss.Slack.bus.v)
bus_load = list(ss.PQ.bus.v)

xcoord_bus_gen = ss.Bus.get(src='xcoord', idx=bus_gen, attr='v')
ycoord_bus_gen = ss.Bus.get(src='ycoord', idx=bus_gen, attr='v')

xcoord_bus_load = ss.Bus.get(src='xcoord', idx=bus_load, attr='v')
ycoord_bus_load = ss.Bus.get(src='ycoord', idx=bus_load, attr='v')

p_bus = gpd.GeoDataFrame(geometry=gpd.points_from_xy(
    ss.Bus.xcoord.v, ss.Bus.ycoord.v))
p_bus_gen = gpd.GeoDataFrame(geometry=gpd.points_from_xy(
    xcoord_bus_gen, ycoord_bus_gen))
p_bus_load = gpd.GeoDataFrame(geometry=gpd.points_from_xy(
    xcoord_bus_load, ycoord_bus_load))

# --- Line Data ---

transmission_lines = []

line_bus1 = ss.Line.get(src='bus1', idx=ss.Line.idx.v, attr='v')
line_bus2 = ss.Line.get(src='bus2', idx=ss.Line.idx.v, attr='v')

for bus1, bus2 in zip(line_bus1, line_bus2):
    bus1x = ss.Bus.get(src='xcoord', idx=bus1, attr='v')
    bus1y = ss.Bus.get(src='ycoord', idx=bus1, attr='v')
    bus2x = ss.Bus.get(src='xcoord', idx=bus2, attr='v')
    bus2y = ss.Bus.get(src='ycoord', idx=bus2, attr='v')
    line_coords = [(bus1x, bus1y), (bus2x, bus2y)]
    transmission_lines.append(LineString(line_coords))

l_lines = gpd.GeoDataFrame(geometry=transmission_lines)

# --- Plotting ---
fig_map, ax_map = plt.subplots(figsize=(10, 6))
plt.ioff()
usa_map.plot(ax=ax_map, edgecolor='black', facecolor='none')

ax, markersize, facecolor = ax_map, 20, 'none'
p_bus_gen.plot(ax=ax, markersize=markersize, facecolor=facecolor, edgecolor='red')
p_bus_load.plot(ax=ax, markersize=markersize, facecolor=facecolor, edgecolor='black')
l_lines.plot(ax=ax, color='black', linewidth=2)

# cx.add_basemap(ax=ax, crs=world.crs.to_string(),
#                source=cx.providers.Stamen.TonerLite)

# Get the extent of the points
marginx = 0.3
marginy = 0.15
x_min, y_min, x_max, y_max = p_bus.total_bounds
x_range = x_max - x_min
y_range = y_max - y_min
ax_map.set_xlim(x_min - marginx * x_range, x_max + marginx * x_range)
ax_map.set_ylim(y_min - marginy * y_range, y_max + marginy * y_range)

ax_map.set_axis_off()

# df_wm = world.to_crs(epsg=3857)
# ax_map = df_wm.plot(ax=ax_map, alpha=0.5, edgecolor='k')

plt.show(block=True)

# plt.show()

In [ ]:
try:
    import geopandas as gpd
    # Read shapefile and plot map
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
    points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(
        ss.Bus.xcoord.v, ss.Bus.ycoord.v))

    fig_map, ax_map = plt.subplots(figsize=(12,6))
    world.plot(ax=ax_map, color='white', edgecolor='black')
    points.plot(ax=ax_map, markersize=100, color='red')
    # Get the extent of the points
    margin = 0.2
    x_min, y_min, x_max, y_max = points.total_bounds
    x_range = x_max - x_min
    y_range = y_max - y_min
    ax_map.set_xlim(x_min - margin * x_range, x_max + margin * x_range)
    ax_map.set_ylim(y_min - margin * y_range, y_max + margin * y_range)
    plt.show()
except ImportError:
    print('Geopandas not found')
